In [1]:
import sys
sys.path.append('..')
from model.modeling_buddygpt import RotaryEmbedding

In [ ]:
from model.modeling_buddygpt import apply_rotary_pos_emb
import torch

rope = RotaryEmbedding(1024)
q, k = torch.randn(1, 16, 16, 1024), torch.randn(1, 16, 16, 1024) # batch_size, seq_len, head, head_dim
cos, sin = rope(q, 16)
q, k = apply_rotary_pos_emb(q, k, cos, sin)
q.shape, k.shape




RuntimeError: The size of tensor a (16) must match the size of tensor b (32) at non-singleton dimension 2